In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from backtesting import Backtest, Strategy

In [3]:
df = pd.read_csv('data/^ixic-60d.csv')
df['Datetime'] = pd.to_datetime(df['Datetime'], utc=True)
df.set_index('Datetime', inplace=True)

In [4]:
df['EMA_50'] = ta.ema(df['Close'], length=50)
df['Sentiment'] = np.where(df['Close'] > df['EMA_50'], 'B', np.where(df['Close'] < df['EMA_50'], 'S', 'N'))
df.ta.macd(fast=12, slow=26, signal=9, append=True)
df['Signal'] = np.where(df['MACDh_12_26_9'] > 0, 'B', np.where(df['MACDh_12_26_9'] < 0, 'S', 'N'))
df['ATR'] = ta.atr(df.High, df.Low, df.Close, length=14)
df = df.dropna(subset=['MACDh_12_26_9', 'ATR', 'EMA_50'])
df.to_csv('data/spgi-60d-ema-macd.csv')
df

Open          High           Low  \
Datetime                                                              
2023-12-27 20:15:00+00:00  15082.018555  15088.772461  15073.826172   
2023-12-27 20:30:00+00:00  15080.443359  15090.893555  15076.179688   
2023-12-27 20:45:00+00:00  15077.320312  15102.096680  15073.445312   
2023-12-28 14:30:00+00:00  15145.951172  15149.165039  15126.885742   
2023-12-28 14:45:00+00:00  15131.515625  15134.285156  15117.427734   
...                                 ...           ...           ...   
2024-03-21 16:30:00+00:00  16507.191406  16508.164062  16488.746094   
2024-03-21 16:45:00+00:00  16490.423828  16497.761719  16484.484375   
2024-03-21 17:00:00+00:00  16495.902344  16495.902344  16475.193359   
2024-03-21 17:15:00+00:00  16486.539062  16486.539062  16470.550781   
2024-03-21 17:30:00+00:00  16474.558594  16475.033203  16466.267578   

                                  Close     Volume  Dividends  Stock Splits  \
Datetime                                                                      
2023-12-27 20:15:00+00:00  15079.820312  156876000        0.0           0.0   
2023-12-27 20:30:00+00:00  15077.727539  210326000        0.0           0.0   
2023-12-27 20:45:00+00:00  15102.096680  336163000        0.0           0.0   
2023-12-28 14:30:00+00:00  15131.934570  203659047        0.0           0.0   
2023-12-28 14:45:00+00:00  15127.285156  242954621        0.0           0.0   
...                                 ...        ...        ...           ...   
2024-03-21 16:30:00+00:00  16490.099609  112714000        0.0           0.0   
2024-03-21 16:45:00+00:00  16495.699219   88439000        0.0           0.0   
2024-03-21 17:00:00+00:00  16486.931641   97042000        0.0           0.0   
2024-03-21 17:15:00+00:00  16476.951172  100175000        0.0           0.0   
2024-03-21 17:30:00+00:00  16471.656250   40659000        0.0           0.0   

                                 EMA_50 Sentiment  MACD_12_26_9  \
Datetime                                                          
2023-12-27 20:15:00+00:00  15069.899902         B      3.684709   
2023-12-27 20:30:00+00:00  15070.206868         B      3.041730   
2023-12-27 20:45:00+00:00  15071.457449         B      4.447285   
2023-12-28 14:30:00+00:00  15073.829101         B      7.878052   
2023-12-28 14:45:00+00:00  15075.925417         B     10.105303   
...                                 ...       ...           ...   
2024-03-21 16:30:00+00:00  16306.043947         B     72.703735   
2024-03-21 16:45:00+00:00  16313.481409         B     70.002887   
2024-03-21 17:00:00+00:00  16320.283379         B     66.389676   
2024-03-21 17:15:00+00:00  16326.427214         B     62.006072   
2024-03-21 17:30:00+00:00  16332.122470         B     57.442616   

                           MACDh_12_26_9  MACDs_12_26_9 Signal        ATR  
Datetime                                                                   
2023-12-27 20:15:00+00:00      -2.104828       5.789537      S  16.310801  
2023-12-27 20:30:00+00:00      -2.198246       5.239976      S  16.193859  
2023-12-27 20:45:00+00:00      -0.634153       5.081438      S  17.104473  
2023-12-28 14:30:00+00:00       2.237291       5.640761      B  19.291114  
2023-12-28 14:45:00+00:00       3.571634       6.533669      B  19.113788  
...                                  ...            ...    ...        ...  
2024-03-21 16:30:00+00:00      -3.080953      75.784688      S  34.777479  
2024-03-21 16:45:00+00:00      -4.625441      74.628328      S  33.241755  
2024-03-21 17:00:00+00:00      -6.590922      72.980597      S  32.346557  
2024-03-21 17:15:00+00:00      -8.779620      70.785692      S  31.206150  
2024-03-21 17:30:00+00:00     -10.674461      68.117077      S  29.740253  

[1502 rows x 14 columns]

In [5]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close']), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['MACD_12_26_9'], mode='lines', name='MACD Line'), row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['MACDs_12_26_9'], mode='lines', name='Signal Line'), row=2, col=1)
fig.add_trace(go.Bar(x=df.index, y=df['MACDh_12_26_9'], name='Histogram'), row=2, col=1)

fig.show()


In [6]:
class MyStrategy(Strategy):
  size = 100
  slCoefficient = 1.1
  tpCoefficient = 1.5

  def init(self):
    super().init()
    self.signal = self.data.Signal

  def next(self):
    super().next()
    # stopLossAtr = self.slCoefficient * (self.data.ATR[-1] if not np.isnan(self.data.ATR[-1]) else 6)
    stopLossAtr = self.slCoefficient * self.data.ATR[-1]

    if self.data['Sentiment'] == 'B' and self.signal == 'B' and len(self.trades) == 0:
      self.position.close()
      stopLoss = self.data.Close[-1] - stopLossAtr
      takeProfit = self.data.Close[-1] + self.tpCoefficient * stopLossAtr
      # self.buy(sl=stopLoss, tp=takeProfit, size=self.size)
      self.buy(sl=stopLoss, size=self.size)
    elif self.data['Sentiment'] == 'S' and self.signal == 'S' and len(self.trades) == 0:
      self.position.close()
      stopLoss = self.data.Close[-1] + stopLossAtr
      takeProfit = self.data.Close[-1] - self.tpCoefficient * stopLossAtr
      # self.sell(sl=stopLoss, tp=takeProfit, size=self.size)
      self.sell(sl=stopLoss, size=self.size)

    # if self.signal == 'B' and len(self.trades) == 0:
    #   stopLoss = self.data.Close[-1] - stopLossAtr
    #   takeProfit = self.data.Close[-1] + self.tpCoefficient * stopLossAtr
    #   self.buy(sl=stopLoss, tp=takeProfit, size=self.size)

    # elif self.signal == 'S' and len(self.trades) == 0:
    #   stopLoss = self.data.Close[-1] + stopLossAtr
    #   takeProfit = self.data.Close[-1] - self.tpCoefficient * stopLossAtr
    #   self.sell(sl=stopLoss, tp=takeProfit, size=self.size)
    # if self.signal == 'B':
    #   stopLoss = self.data.Close[-1] - stopLossAtr
    #   takeProfit = self.data.Close[-1] + self.tpCoefficient * stopLossAtr
    #   self.buy(sl=stopLoss, tp=takeProfit, size=self.size)

    # elif self.signal == 'S':
    #   stopLoss = self.data.Close[-1] + stopLossAtr
    #   takeProfit = self.data.Close[-1] - self.tpCoefficient * stopLossAtr
    #   self.sell(sl=stopLoss, tp=takeProfit, size=self.size)


bt = Backtest(df, MyStrategy, cash=1000000000, commission=.002)

results = bt.run()
bt.plot()
# Print the backtesting results
results['_equity_curve'].to_csv('data/^ixic-60d-60d-equity_curve.csv')
results['_trades'].to_csv('data/^ixic-60d-60d-trades.csv')
results


/Users/andrew/.pyenv/versions/3.11.4/lib/python3.11/site-packages/backtesting/_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

/Users/andrew/.pyenv/versions/3.11.4/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

/Users/andrew/.pyenv/versions/3.11.4/lib/python3.11/site-packages/backtesting/_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

/Users/andrew/.pyenv/versions/3.11.4/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

/Users/andrew/.pyenv/versions/3.11.4/lib/python3.11/site-packages/backtesting/

Start                     2023-12-27 20:15...
End                       2024-03-21 17:30...
Duration                     84 days 21:15:00
Exposure Time [%]                    58.85486
Equity Final [$]             999815797.221458
Equity Peak [$]             1000053382.238181
Return [%]                           -0.01842
Buy & Hold Return [%]                9.229791
Return (Ann.) [%]                   -0.078653
Volatility (Ann.) [%]                0.022562
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.023757
Avg. Drawdown [%]                   -0.002313
Max. Drawdown Duration       75 days 21:15:00
Avg. Drawdown Duration        6 days 09:19:00
# Trades                                   25
Win Rate [%]                              0.0
Best Trade [%]                      -0.200401
Worst Trade [%]                     -1.153878
Avg. Trade [%]                    